In [1]:
import numpy as np
import pandas as pd

In [2]:
df_train = pd.read_csv('/Users/dennis_m_jose/Documents/GitHub/Santander-RecSys-Dennis/data/cleaned_santander_data.csv')

/var/folders/jl/5_rcdjz121v7rnhqrmsz6df80000gn/T/ipykernel_72266/1706210101.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv('/Users/dennis_m_jose/Documents/GitHub/Santander-RecSys-Dennis/data/cleaned_santander_data.csv')


## BASELINE MODELLING

In [3]:
df_train[df_train['fecha_dato'] < '2016-05-01']

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-01-28,1375586,N,ES,H,35.0,2015-01-12,0.0,6,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
1,2015-01-28,1050611,N,ES,V,23.0,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
2,2015-01-28,1050612,N,ES,V,23.0,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
3,2015-01-28,1050613,N,ES,H,22.0,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
4,2015-01-28,1050614,N,ES,V,23.0,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12687899,2016-04-28,1297410,N,ES,V,47.0,2014-08-18,0.0,20,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
12687900,2016-04-28,1297411,N,ES,H,41.0,2014-08-18,0.0,20,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
12687901,2016-04-28,1297412,N,ES,V,34.0,2014-08-18,0.0,20,1.0,...,0,0,0,0,0,0,0,0.0,0.0,1
12687902,2016-04-28,1297392,N,ES,H,44.0,2014-08-18,0.0,20,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [4]:
df_train[df_train['fecha_dato'] >= '2016-05-01']['fecha_dato'].unique()

array(['2016-05-28'], dtype=object)

In [5]:

# Split: train on data before May 2016, validate on May 2016
train_df = df_train[df_train['fecha_dato'] < '2016-05-01']
eval_df = df_train[df_train['fecha_dato'] == '2016-05-28']

In [6]:
product_cols = [col for col in df_train.columns if col.endswith('ult1')]

In [8]:
# Sum of additions per product
product_popularity = train_df[product_cols].sum().sort_values(ascending=False)

# Get top 7 products
top_7_products = product_popularity.head(7).index.tolist()
print(top_7_products)

['ind_cco_fin_ult1', 'ind_ctop_fin_ult1', 'ind_recibo_ult1', 'ind_ecue_fin_ult1', 'ind_cno_fin_ult1', 'ind_nom_pens_ult1', 'ind_nomina_ult1']


In [9]:
# Get unique customers in May 2016
eval_users = eval_df['ncodpers'].unique()

# Predict same top 7 for all
baseline_preds = pd.DataFrame({
    'ncodpers': eval_users,
    'added_products': [' '.join(top_7_products)] * len(eval_users)
})

In [10]:
def apk(actual, predicted, k=7):
    if len(predicted) > k:
        predicted = predicted[:k]

    if not actual:
        return 0

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    return score / min(len(actual), k)

def mapk(actual_list, predicted_list, k=7):
    return np.mean([apk(a, p.split(), k) for a, p in zip(actual_list, predicted_list)])


In [11]:
# Ground truth from eval_df
eval_truth = []
for _, row in eval_df.groupby('ncodpers'):
    added = [prod for prod in product_cols if row[prod].values[0] == 1]
    eval_truth.append(added)

# Compute MAP@7
map_score = mapk(eval_truth, baseline_preds['added_products'].tolist())
print(f"Baseline MAP@7: {map_score:.4f}")

Baseline MAP@7: 0.5937


In [12]:
baseline_preds

,ncodpers,added_products
0,657640,ind_cco_fin_ult1 ind_ctop_fin_ult1 ind_recibo_...
1,657788,ind_cco_fin_ult1 ind_ctop_fin_ult1 ind_recibo_...
2,657795,ind_cco_fin_ult1 ind_ctop_fin_ult1 ind_recibo_...
3,657790,ind_cco_fin_ult1 ind_ctop_fin_ult1 ind_recibo_...
4,657794,ind_cco_fin_ult1 ind_ctop_fin_ult1 ind_recibo_...
...,...,...
931448,1166765,ind_cco_fin_ult1 ind_ctop_fin_ult1 ind_recibo_...
931449,1166764,ind_cco_fin_ult1 ind_ctop_fin_ult1 ind_recibo_...
931450,1166763,ind_cco_fin_ult1 ind_ctop_fin_ult1 ind_recibo_...
931451,1166789,ind_cco_fin_ult1 ind_ctop_fin_ult1 ind_recibo_...


### Error analysis

In [13]:
# Convert predictions into list of products
baseline_preds['prediction_list'] = baseline_preds['added_products'].apply(lambda x: x.split())

# Create a comparison dataframe
comparison = pd.DataFrame({
    "ncodpers": baseline_preds['ncodpers'],
    "actual": eval_truth,
    "predicted": baseline_preds['prediction_list']
})


In [14]:
#  False Positives & False Negatives
comparison['false_positives'] = comparison.apply(
    lambda row: [p for p in row['predicted'] if p not in row['actual']], axis=1)

comparison['false_negatives'] = comparison.apply(
    lambda row: [a for a in row['actual'] if a not in row['predicted']], axis=1)

In [15]:

# Count most common FP/FN products
fp_counts = pd.Series([p for row in comparison['false_positives'] for p in row]).value_counts()
fn_counts = pd.Series([p for row in comparison['false_negatives'] for p in row]).value_counts()

print("\nTop False Positives (Over-predicted products):")
print(fp_counts.head(10))

print("\nTop False Negatives (Missed products):")
print(fn_counts.head(10))


Top False Positives (Over-predicted products):
ind_nomina_ult1      883144
ind_nom_pens_ult1    878605
ind_cno_fin_ult1     858392
ind_ecue_fin_ult1    855608
ind_ctop_fin_ult1    830864
ind_recibo_ult1      818403
ind_cco_fin_ult1     369838
dtype: int64

Top False Negatives (Missed products):
ind_reca_fin_ult1    45535
ind_tjcr_fin_ult1    34822
ind_ctpp_fin_ult1    33271
ind_dela_fin_ult1    31217
ind_valo_fin_ult1    21426
ind_fond_fin_ult1    14687
ind_ctma_fin_ult1     8097
ind_ctju_fin_ult1     7581
ind_plan_fin_ult1     7359
ind_hip_fin_ult1      4528
dtype: int64


### Collaborative Filtering